# Linear Regression using La Classy

In [17]:
import {
  OLSSolver
} from "jsr:@lala/classy@1.2.2";

import { r2, rmse, Matrix } from "jsr:@lala/appraisal@0.7.5";

Let's initialize the predictor and target variables.

We will make our model learn the function `y1 = 9x1 + 2x2 + 5x3 + 8`, `y2 = 3x1 + 4x2 + 3x3 + 5`.

In [18]:
const X = [[5, 6, 10], [9, 13, 2], [12, 45, 92], [8, 2, 13], [43, 4, 19]]
const y = X.map(x => ([x[0] * 9 + x[1] * 2 + x[2] * 5 + 8, x[0] * 3 + x[1] * 4 + x[2] * 3 + 5]))

y

[ [ 115, 74 ], [ 125, 90 ], [ 666, 497 ], [ 149, 76 ], [ 498, 207 ] ]

In [19]:
const X_train = new Matrix(X, "f64");
const y_train = new Matrix(y, "f64");

Now that we have prepared our inputs, we can initialize our solver. Since we are performing linear regression, let's try out an Ordinary Least Squares solver.

In [20]:
const solver = new OLSSolver();

We can then train our model using the data we acquired.

In [21]:
const time = performance.now();
solver.train(X_train, y_train, {
  fit_intercept: true,
});
console.log(`training time: ${performance.now() - time}ms`);

training time: 0.08760000002803281ms


The model is trained, now it is time to evaluate its performance on our testing dataset

In [22]:
const res = solver.predict(X_train)
res.shape

[ 5, 2 ]

Check R-squared score (should be close to 1)

In [23]:
r2(res.data, y_train.data)

0.9999999999999966

Check Root Mean Square Error (smaller = better)

In [24]:
rmse(res.data, y_train.data)

5.908708654470939e-13